In [103]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def get_daily_data(f):
    df = pd.read_csv(f,sep=',')
    us = df.groupby('Country_Region').get_group('US')
    ny_mask = us.loc[:,'Province_State'] == 'New York'
    ny = us[ny_mask].copy()
    ny['Last_Update'] = pd.to_datetime(ny['Last_Update'])
    ny['Last_Update'] = ny['Last_Update'].dt.date
    return ny

def get_daily_data_2(f):
    df = pd.read_csv(f,sep=',')
    us = df.groupby('Country/Region').get_group('US')
    ny_mask = us.loc[:,'Province/State'] == 'New York'
    ny = us[ny_mask].copy()
    ny['Last Update'] = pd.to_datetime(ny['Last Update'])
    ny['Last Update'] = ny['Last Update'].dt.date
    return ny

f = '~/corona/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-29-2020.csv'

df = get_daily_data(f)


df2 = get_daily_data_2('~/corona/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-21-2020.csv')

In [105]:
import glob

file_prefix = '../../corona/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv'

files = [f for f in glob.glob(file_prefix)]
files = sorted(files)

file_s = pd.Series(files)
change_idx = 59

timeline_df_1 = pd.DataFrame()
timeline_df_2 = pd.DataFrame()

for i,f in file_s.iteritems():
    
    
    if i > change_idx:
        temp = get_daily_data(f)
        timeline_df_1 = pd.concat([timeline_df_1,temp],axis=0)
    else:
        temp = get_daily_data_2(f)
        timeline_df_2 = pd.concat([timeline_df_2,temp],axis=0)
        
    
timeline_df_1 = timeline_df_1.groupby(['Province_State','Last_Update']).sum()

timeline_df_1.drop(['FIPS','Lat','Long_','Active'],axis=1,inplace=True)
timeline_df_1.reset_index(inplace=True)
timeline_df_2.drop(['Country/Region','Latitude','Longitude'],axis=1,inplace=True)
timeline_df_2.rename(columns={'Province/State':'Province_State',
                             'Last Update':'Last_Update'},inplace=True)

timeline_df = pd.concat([timeline_df_2,timeline_df_1],axis=0)
#timeline_df.drop(203,inplace=True) # double entry
timeline_df.at[203,'Last_Update'] = pd.to_datetime('2020-03-13').date() # wrong date

timeline_df['inc'] = timeline_df['Confirmed'] - timeline_df['Confirmed'].shift()
timeline_df['inc_dead'] = timeline_df['Deaths'] - timeline_df['Deaths'].shift()
timeline_df['factor'] = timeline_df['Confirmed'] / timeline_df['Confirmed'].shift()
timeline_df['factor_dead'] = timeline_df['Deaths'] / timeline_df['Deaths'].shift()
timeline_df.replace(np.inf,np.nan,inplace=True)

timeline_df.columns = ['state','date','confirmed','dead','recovered','inc','inc_dead',
                       'factor','factor_dead']

timeline_df.to_pickle('ny_timeline.pkl')
timeline_df

,state,date,confirmed,dead,recovered,inc,inc_dead,factor,factor_dead
36,New York,2020-03-10,173.0,0.0,0.0,NaN,NaN,NaN,NaN
36,New York,2020-03-11,220.0,0.0,0.0,47.0,0.0,1.271676,NaN
29,New York,2020-03-12,328.0,0.0,0.0,108.0,0.0,1.490909,NaN
203,New York,2020-03-13,421.0,0.0,0.0,93.0,0.0,1.283537,NaN
28,New York,2020-03-14,525.0,2.0,0.0,104.0,2.0,1.247031,NaN
23,New York,2020-03-15,732.0,3.0,0.0,207.0,1.0,1.394286,1.500000
19,New York,2020-03-16,967.0,10.0,0.0,235.0,7.0,1.321038,3.333333
9,New York,2020-03-17,1706.0,13.0,0.0,739.0,3.0,1.764219,1.300000
9,New York,2020-03-18,2495.0,16.0,0.0,789.0,3.0,1.462485,1.230769
7,New York,2020-03-19,5365.0,34.0,0.0,2870.0,18.0,2.150301,2.125000
